<center><h1> Save Data to Geo File </h1></center>

## Load data

In [1]:
import pandas as pd
import json

df = pd.read_csv('listings.csv')
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [2]:
import json
keywords = json.load(open("keywords.txt"))

In [3]:
pred = pd.read_csv('pred_price.csv')
pred_dict = pred[['id', 'pred']].set_index('id').to_dict().get('pred')

## Function to convert to GeoJsn for each host

In [4]:
# http://geoffboeing.com/2015/10/exporting-python-data-geojson/

def df_to_geojson(df, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        predicted = pred_dict.get(row['id'],'No Prediction') 
        try:
            kw = str(keywords.get(str(row['id']), 'No Reviews'))  
        except:
            kw = 'No Reviews'
        description = '<strong>' + str(row['name']) + '</strong><p><a href=\"' + str(row['listing_url']) + \
                      '" target=\"_blank\" title=\"Go to Airbnb to Order\">Airbnb Link for Host</a><br> <strong>Real Price </strong>: ' \
                      + str(row['price']) + '<br><strong> Predicted Price (Value) </strong>: $' + \
                      str(predicted) + '<br><strong> Keywords in Reviews: </strong>' + kw + '</p>'
        feature = {'type':'Feature',
                   'properties':{'description': description},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        geojson['features'].append(feature)
    return geojson

## Save file

In [5]:
geojson = df_to_geojson(df)

In [6]:
output_filename = 'hosts.geojson'
with open(output_filename, 'wb') as output_file:
    #output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    
print('Done') 

Done
